In [ ]:
from pathlib import Path
import sys

lib_path = str(Path('../').resolve())
sys.path.insert(0, lib_path)
lib_path

In [ ]:
from crops import CropModels
from utils.weather import get_weather_df


csv_fp = Path('../../../assets') / 'sample.csv'

weather_df = get_weather_df(csv_fp)
crop_models = {ModelClass.key: ModelClass for ModelClass in CropModels}


### 배추
  - 한계온도 & 노출일수
    - high_extrema_temperature = 30
    - high_extrema_exposure_days = 5
    - low_extrema_temperature = 5
    - low_extrema_exposure_days = 5

In [ ]:
crop_model = crop_models.get('springCabbage')()

crop_model.set_weather_data(weather_df)
df = crop_model.gdd_weather_df.copy()

crop_model.high_extrema_temperature = 25  # DEV
crop_model.low_extrema_temperature = 5  # DEV

# 생육한계 최고온도 & 노출일수
df['high_extrema_exposure'] = df['tmax'] > crop_model.high_extrema_temperature
df['high_extrema_exposure_group'] = df['high_extrema_exposure'].diff(1).cumsum()

high_extrema_exposure_doy_ranges = []
for idx, group in df.groupby('high_extrema_exposure_group'):
    high_extrema_col = group['high_extrema_exposure']

    if high_extrema_col.all() \
      and high_extrema_col.count() > crop_model.high_extrema_exposure_days:

      high_extrema_exposure_doy_ranges.append(
        [group.index[0], group.index[-1]]
      )

# 생육한계 최저온도 & 노출일수
df['low_extrema_exposure'] = df['tmin'] < crop_model.low_extrema_temperature
df['low_extrema_exposure_group'] = df['low_extrema_exposure'].diff(1).cumsum()

low_extrema_exposure_doy_ranges = []
for idx, group in df.groupby('low_extrema_exposure_group'):
    low_extrema_col = group['low_extrema_exposure']

    if low_extrema_col.all() \
      and low_extrema_col.count() > crop_model.low_extrema_exposure_days:

      low_extrema_exposure_doy_ranges.append(
        [group.index[0], group.index[-1]]
      )

In [ ]:
display(high_extrema_exposure_doy_ranges)
display(low_extrema_exposure_doy_ranges)